In [3]:
import os
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [ ]:
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://en.wikipedia.org/wiki/MongoDB")
documents = loader.load()
documents

USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'https://en.wikipedia.org/wiki/MongoDB', 'title': 'MongoDB - Wikipedia', 'language': 'en'}, page_content='\n\n\n\nMongoDB - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload fileSpecial pages\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\nHistory\n\n\n\n\n\n\n

In [5]:
#spliters
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_documents(documents)
chunks

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/MongoDB', 'title': 'MongoDB - Wikipedia', 'language': 'en'}, page_content='MongoDB - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload fileSpecial pages\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\nHistory\n\n\n\n\n\n\n\n\n2\nB

In [6]:
#embeddings
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="nomic-embed-text")

res = embeddings.embed_documents([chunk.page_content for chunk in chunks])



In [7]:
#store in vector DB chromadb
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_documents(chunks, embeddings)
vectorstore

In [ ]:
#querying
query = "What is MongoDB?"
docs = vectorstore.similarity_search(query)
docs

[Document(metadata={'title': 'MongoDB - Wikipedia', 'source': 'https://en.wikipedia.org/wiki/MongoDB', 'language': 'en'}, page_content='MongoDBDeveloperMongoDB Inc.Initial releaseFebruary\xa011, 2009; 16 years ago\xa0(2009-02-11)[1]Stable release8.1.2\xa0\n   / 2 July 2025, 3 months ago\nRepositorygithub.com/mongodb/mongo \nWritten inC++, JavaScript, Python, COperating systemWindows Vista and later, Linux, OS X 10.7 and later, Solaris,[2] FreeBSD[3]Available inEnglishTypeDocument-oriented databaseLicenseServer Side Public License or proprietaryWebsitemongodb.com\nMongoDB is a source-available, cross-platform, document-oriented database program. Classified as a NoSQL database product, MongoDB uses JSON-like documents with optional schemas. Released in February 2009 by 10gen (now MongoDB Inc.), it supports features like sharding, replication, and ACID transactions (from version 4.0). MongoDB Atlas, its managed cloud service, operates on AWS, Google Cloud Platform, and Microsoft Azure. Cu

In [36]:
retriever = vectorstore.as_retriever()

from langchain.llms import Ollama
llm = Ollama(model="gemma3:1b", temperature=0.1)

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

pro = ChatPromptTemplate.from_template("""
You are a helpful assistant that helps people. Based on the context {context}, answer the question: {input}
""")

document_chain = create_stuff_documents_chain(llm, pro)


In [37]:
# Step 5: Create the retrieval chain
from langchain.chains import create_retrieval_chain
rag_chain = create_retrieval_chain(retriever, document_chain)


In [ ]:
question = "When mongoDB was released?  What is its primary use?"
response = rag_chain.invoke({"input": question})
print(response["answer"])

Okay, here's a breakdown of the key information from the text, formatted for clarity:

**MongoDB Overview**

*   **Release History:**
    *   February 11, 2009: Initial Release
    *   August 2009: Feature Release
    *   December 2009: Feature Release
    *   2013: Name Change to MongoDB Inc.
    *   October 20, 2017: Publicly Traded (NASDAQ: MDB)
    *   November 8, 2018: License Change to SSPL (Server Side Public License)
    *   October 30, 2019: Partnership with Alibaba Cloud for a MongoDB-as-a-Service offering.
*   **License:** Server Side Public License (SSPL)
*   **Primary Use:**  Manages large databases of unstructured, "messy" data. Primarily used for mobile and web applications that frequently use unstructured databases.
*   **Current Status:** Currently licensed under the SSPL.

**Key Facts**

*   **Released in February 2009.**
*   **Originally a NoSQL database product.**
*   **Atlas (2016) became the primary revenue source.**
*   **Originally free and open source (AGPL 3.0